In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time # for sleep time
import re
import functools
import pandas as pd
driver = webdriver.Chrome(executable_path=r'C:\Users\Xitong Hu\Desktop\Class\BAN5193\chromedriver.exe')

## Import the URL list

In [2]:
df=pd.read_csv("Chicago_hotel_list.csv")

In [3]:
df

,url
0,https://www.hotels.com/ho106418/?q-check-out=2...
1,https://www.hotels.com/travelads/trackredirect...
2,https://www.hotels.com/travelads/trackredirect...
3,https://www.hotels.com/ho148843/?q-check-out=2...
4,https://www.hotels.com/ho422531/?pa=5&tab=desc...
...,...
461,https://www.hotels.com/ho1480392704/?pa=464&ta...
462,https://www.hotels.com/ho537680/?pa=465&tab=de...
463,https://www.hotels.com/ho474678/?pa=466&tab=de...
464,https://www.hotels.com/ho311002/?pa=467&tab=de...


### Start to scrap review from a sample url, hotel_all_links[0]. This section shows the some process of how to extract review features from one URL. 

In [4]:
#hotel_all_links[0]

In [5]:
#driver.get(hotel_all_links[0])

In [6]:
#After open the hotel page, we need to click the see all review button to read more reviews.
#See_all_reviews_button = driver.find_elements_by_xpath('//div[@class="see-all-reviews"]/a[@class="cta"]')

In [7]:
#See_all_reviews_button[0].click()

In [8]:
#Scrape the review features in the first review page
#Review=driver.find_elements_by_xpath('//*[@id="guest-reviews"]/div[2]/div[1]/div[3]/div[2]/div[*]/blockquote')

### Write loop to exctract reviews from 100 hotel urls. For each hotel, we are going to extract at max 100 most recent reviews.

In [10]:
#Subset the whole url list to small lists.
hotel_all_links=df["url"]
hotel_1=hotel_all_links[0:20]
hotel_1

0     https://www.hotels.com/ho106418/?q-check-out=2...
1     https://www.hotels.com/travelads/trackredirect...
2     https://www.hotels.com/travelads/trackredirect...
3     https://www.hotels.com/ho148843/?q-check-out=2...
4     https://www.hotels.com/ho422531/?pa=5&tab=desc...
5     https://www.hotels.com/ho218070/?q-check-out=2...
6     https://www.hotels.com/travelads/trackredirect...
7     https://www.hotels.com/travelads/trackredirect...
8     https://www.hotels.com/ho482991/?q-check-out=2...
9     https://www.hotels.com/ho122910/?q-check-out=2...
10    https://www.hotels.com/ho421352/?pa=11&tab=des...
11    https://www.hotels.com/ho1462835840/?q-check-o...
12    https://www.hotels.com/ho862142112/?q-check-ou...
13    https://www.hotels.com/ho265809/?q-check-out=2...
14    https://www.hotels.com/ho310742/?q-check-out=2...
15    https://www.hotels.com/ho699797/?q-check-out=2...
16    https://www.hotels.com/ho126022/?q-check-out=2...
17    https://www.hotels.com/travelads/trackredi

In [12]:
#So in this scraping process, it only shows the process for the first 20 url links.
#We need to muanlly change the slice number to create another subset until we scrape 100 url links.
#such as:
#url20_50=list(hotel_all_links[20:50])

In [11]:
#create empty list to scrap reviews
Rate_list=[]
Review_id=[]
Date=[]
Reviews=[]
URL_ID=[]
Hotel_Name=[]
error=""

In [12]:
for url in hotel_1: #change the hotel_1 to other subset names when process the next group.
    #get open the url in the selenium browser.
    driver.get(url)
    #pring the current url for checing the loop process
    print(url)
    #We need to add some sleep seconds after open the url page.
    time.sleep(15)
    #Srape the title of the hotel in the main page
    title=driver.find_element_by_xpath('//div[@class="vcard"]/h1').text
    #Find the see all reviews button at the main page and click it. Sometimes, a hotel might not have reveiw information.
    #In such condition, an error will occure. 
    #So we use try except continue to supress the error and continue to next url,if a page does not have see all review button.
    try:
        See_all_reviews_button = driver.find_elements_by_xpath('//div[@class="see-all-reviews"]/a[@class="cta"]')
        See_all_reviews_button[0].click()
    except:
        print(url+"error")
        continue

    c = 0
    time.sleep(2)
    
    #Get the total review number for the hotel
    Total_Review=driver.find_element_by_xpath('//*[@id="guest-reviews"]/div[2]/div[1]/div[1]/ul/li[1]/a/span').text
    Total_Review_num=int(re.sub(r"[()]", "", Total_Review))
    
    #since we only scrape the first 100 reviews for a hotel, so we need to loop 2 times to get 100 reviews (50 reviews per page)
    while c < 2:
        #for some reviews, partial conents were shown, we need to click the expanable button to show the whole comments.
        #click the expanable_buttons to show the whole comments
        Expanable_buttons=driver.find_elements_by_xpath('//div[@class="expandable-wrapper"]/span/button')
        try: # not all pages have buttons
            for a in range(0, len(Expanable_buttons)):
                Expanable_buttons[a].click()
                time.sleep(20)
        except:
               error="no_read_more"
        #start to write a inside loop to extract review features one by one. 
        #Because a review does not alway include all features, so we need to etract them one by one for their features.
        
        #start the inside loop, the maximum review amounts in a page is equal to the rating occurence in the review list.
        #we can set it to maximum 50 per review page, but it will raise error if a page includes less than 50 reviews. 
        Rate_len=driver.find_elements_by_xpath('//*[@id="guest-reviews"]/div[2]/div[1]/div[3]/div[2]/div[*]/div[1]/span[1]/span[1]')
        for x in range(1,len(Rate_len)+1):
            time.sleep(2)
            #Get the rating of one review, if error occures, assign "NA".
            try:
                Rate=driver.find_element_by_xpath('//*[@id="guest-reviews"]/div[2]/div[1]/div[3]/div[2]/div[%d]/div[1]/span[1]/span[1]' %x) 
                Rate_list.append(Rate.text)
            except:
                Rate_list.append("NA")
            #Get the date of one review, if error occures, assign "NA".
            try:
                Date_1=driver.find_element_by_xpath('//*[@id="guest-reviews"]/div[2]/div[1]/div[3]/div[2]/div[%d]/div[1]/span[2]' %x)
                Date.append(Date_1.text)
            except:
                Date.append("NA")
            #Write the url inforamtion for this review
            URL_ID.append(url)
            #Write the hotel name for this reveiw
            Hotel_Name.append(title)
            #check the progress when loop is runing. It shows which comment we are currently scraping.
            print(c*50+x)
            #Scrape the review
            try:
                Review=driver.find_element_by_xpath('//*[@id="guest-reviews"]/div[2]/div[1]/div[3]/div[2]/div[%d]/blockquote' %x)
                Reviews.append(Review.text)
            except:
                #sometimes, even the xpath is correct, it also raises an error. So I make it sleep 10 seconds and try it again.
                try:
                    time.sleep(10)
                    Review=driver.find_element_by_xpath('//*[@id="guest-reviews"]/div[2]/div[1]/div[3]/div[2]/div[%d]/blockquote' %x)
                    Reviews.append(Review.text)
                except:
                    #there's another xpath for the review content
                    try:
                        Review=driver.find_element_by_xpath('//*[@id="guest-reviews"]/div[2]/div[1]/div[3]/div[2]/div[%d]/div[2]/blockquote' %x)
                        Reviews.append(Review.text)
                    except:
                        #if above conditions are not met, assign "Error" for this review.
                        Review="Error"
                        Reviews.append(Review)
                        pass
                
        #After scraping the first 50 reviews, we need to click next button to the next review page.
        #There are two possible xpaths for next_button.
        #time.sleep(10)
        try:
            next_button=driver.find_element_by_xpath('//div[@class="pagination-controls"]/a[2]')
            next_button.click()
        except:
            try:
                next_button_first_page=driver.find_element_by_xpath('//*[@id="guest-reviews"]/div[2]/div[1]/div[3]/div[2]/div[51]/div[2]/a')
                next_button_first_page.click()
            except:
                pass
        #Add conditions to avoid some errors, for example, if a review page does not have read more option in the first page,
        #an error will occure and it will make the loop to the next page to scrape features. 
        #So we add conditions to make it in the loop even there's no read more option for reviews.
        if error == "no_read_more" and Total_Review_num >50 and Rate == "":
            c=c
        elif Total_Review_num <50:
            c=c+2
        else:
            c+=1

https://www.hotels.com/ho106418/?q-check-out=2020-10-27&FPQ=2&q-check-in=2020-10-26&WOE=2&WOD=1&q-room-0-children=0&pa=1&tab=description&JHR=1&q-room-0-adults=2&YGF=2&MGT=1&ZSX=0&SYE=3
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
https://www.hotels.com/travelads/trackredirect.html?trackingUrl=H4sIAAAAAAAAAC2U266qOhiF38arhbPlVFiJ2QERURA5iegNAVrlDAoC8vSbPbN70ZFv5B9NRtM07fu2-_vzM47jmjwjPJB3n3VZ_VwnTfXTv6OBlBLufgb44_0P2zJLin960vU-eXdZU5-X0DvDZMOKPFyDPyJglx1xv4BoyK3hH4GGwkICgnBN_xFEWlgzyyD4NRkWLeZC_-UEkaOXABJF-ndyiS9HIlpcTBFB_j8SAPMrNPsr_CKrd1QXG7hKohpnOOqJVu0_Gd6wEc0TwBEqEgBNsRESqYjlGCpiBC7BTMxFDLNaeibFUlqJ-mgjLWun7TXv3MuJ2TFH5OeHaxvehYmnfB2S206laZkdBR09lJiDSRYlVSfrWUmlkWcleIq3KT-_M0ejM4-fchzH24Eb7cNx1jJtW6XKHKSTQvaZpdTp2H2r4dkJbmub4iN60PNY21lFj9f

KeyboardInterrupt: 

### Build a dataframe based on the attributes we collected

In [158]:
df = pd.DataFrame(list(zip(URL_ID,Hotel_Name,Date, Rate_list, Reviews)), 
               columns =['URL','Hotel_Name','Date', 'Rate','Review']) 

In [160]:
#export to the csv file
df.to_csv("List1.csv",index=False)

### This section shows all the subset csv files of the first 100 reviews.

In [13]:
df1=pd.read_csv("List1.csv")
df2=pd.read_csv("List2_10.csv")
df3=pd.read_csv("List11_30.csv")
df4=pd.read_csv("List31_45.csv")
df5=pd.read_csv("List46_59.csv")
df6=pd.read_csv("List60_63.csv")
df7=pd.read_csv("List64_99.csv")
df8=pd.read_csv("List100.csv")

In [14]:
#append into one dataframe and delete the duplicates. My loop function scrapes the reviews twice under certain error condition.
#due to the limit of the time, haven't fix this bug. But it does not effect the data quality.
df_final=df1.append(df2).append(df3).append(df4).append(df5).append(df6).append(df7).append(df8).drop_duplicates()

In [15]:
df_final

,URL,Hotel_Name,Date,Rate,Review
0,https://www.hotels.com/ho106418/?q-check-out=2...,The Whitehall Hotel,"Check-in Oct 19, 2020",8.0,It was a good stay there
1,https://www.hotels.com/ho106418/?q-check-out=2...,The Whitehall Hotel,"Check-in Oct 20, 2020",4.0,Very disappointing. Hallway carpets very old a...
2,https://www.hotels.com/ho106418/?q-check-out=2...,The Whitehall Hotel,"Check-in Oct 18, 2020",4.0,"Cheap room, not clean. Water damage EVERYWHERE..."
3,https://www.hotels.com/ho106418/?q-check-out=2...,The Whitehall Hotel,"Check-in Oct 17, 2020",10.0,The staff was extremely accommodating and nice...
4,https://www.hotels.com/ho106418/?q-check-out=2...,The Whitehall Hotel,"Check-in Oct 17, 2020",6.0,The comforter was disgusting. We didn’t realiz...
...,...,...,...,...,...
28,https://www.hotels.com/ho1168217056/?q-check-o...,Nobu Hotel Chicago,"Check-in Aug 12, 2020",10.0,NaN
29,https://www.hotels.com/ho1168217056/?q-check-o...,Nobu Hotel Chicago,"Check-in Jul 9, 2020",8.0,NaN
30,https://www.hotels.com/ho1168217056/?q-check-o...,Nobu Hotel Chicago,"Check-in Jul 3, 2020",10.0,NaN
31,https://www.hotels.com/ho1168217056/?q-check-o...,Nobu Hotel Chicago,"Check-in Jul 3, 2020",8.0,NaN


In [16]:
#the number of distinct URL
len(df_final.URL.unique())

100

In [17]:
df_final.to_csv("Reviews_100_URL.csv",index=False)